In [1]:
import sys, platform, os
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import scipy as sp
import scipy.special
import random
from mpmath import mp
import time
import matplotlib.patches as mpatches

from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import Axes3D



In [2]:
def Bessel(x,l):
    return sp.special.spherical_jn(l,x)


In [3]:
mp.dps = 200 # accurate up to 100 decimal places. The accuracy can be switched here.

In [4]:
def FillFacTuple(x_max,Prefactor = mp.mpf(1.),Exponent = 0):
    # returns the faculties calculated up to x_max. More specific: Returns A,B in the form fac(x) = A* (10**6) **B
    #WATCH OUT: x has to be integer!!!
    
    if(x_max != int(x_max)):
        print("X_MAX HAS TO BE AN INTEGER")
    
    x = x_max #oops
    
    FacTuple =[-1]*(x_max*2+2)
    
    x = int(x)
    
    for i in range(1,x+1):
        
        if(Prefactor*i < 10**6):
            Prefactor *= i
            FacTuple[2*i] = np.array([Prefactor, Exponent])
        
        else:
            Prefactor = Prefactor /mp.mpf(10**6)
            Prefactor *=i
            Exponent += 1
            FacTuple[2*i] = np.array([Prefactor, Exponent])
            
            
    FacTuple[0] = np.array([mp.mpf(1.),0.])
    
    x = x+0.5
    
    Prefactor = mp.sqrt(mp.pi)/2
    Exponent = 0
    for i in range(3, int(2*x)+1,2):

        if(Prefactor*i < 10**6):
            Prefactor *= i/2
            FacTuple[i] = np.array([Prefactor, Exponent])
        else:
            Prefactor = Prefactor /mp.mpf(10**6)
            Prefactor *=i/2
            Exponent += 1
            FacTuple[i] = np.array([Prefactor, Exponent])

    FacTuple[1] = np.array([mp.sqrt(mp.pi)/2,0])
    
    return FacTuple

FacTuple = np.array(FillFacTuple(7000))

In [5]:
def fac(x):

    if(type(x) == np.ndarray):
        y = (2*x).astype(int)
        return FacTuple[y].transpose()
    
    return FacTuple[int(2*x)]



In [6]:
def Reduced_E(a,b,c): #so far only suitable for scalar a,b,c.

    a1,b1 = fac(a+b+c+1)
    a2,b2 = fac(1/2.*(a+b-c))
    a3,b3 = fac(1/2.*(a-b+c))
    a4,b4 = fac(1/2.*(-a+b+c))

    a5,b5 = fac(a+b-c)
    a6,b6 = fac(1/2.*(a+b+c))

    return np.array([a1*a2*a3*a4/(a5*a6),b1+b2+b3+b4-b5-b6])

In [7]:
def Triangle(a,b,c): #Okay, these functions are stated here, but they are not actually used.
    if((a+b+c)%2==0):
        if(abs(a-b)<=c<=a+b):
            if(abs(b-c)<=a<=b+c):
                if(abs(a-c)<=b<=a+c):
                    return 1
    return 0
    
def WeakTriangle(a,b,c):    
    if(abs(a-b)<=c<=a+b):
        if(abs(b-c)<=a<=b+c):
            if(abs(a-c)<=b<=a+c):
                return 1
    return 0
    
    
def E(a,b,c):
    
    if(Triangle(a,b,c)==0):
        
        return mp.mpf(1)* np.array([0,0])
    
    return Reduced_E(a,b,c)

def Teta(k1,k2,k3):
    
    if(abs(k1-k2)<k3<abs(k1+k2)):
        return 1
    
    if(abs(k1-k2)==k3 or abs(k1+k2)==k3):
        return 0.5
    
    return 0

In [8]:
def Q_Modified(l3,L,k1,l1,k2,l2):
    
    if(Triangle(l2,l1,l3)==0): 
                
        return mp.mpf(0)

    
    a_pre,b_pre = E(l2,l1,l3)

    prefactor = a_pre*(-1)**(int((l3-l1+l2)/2))*(2*L+1)/2

    #here I add the terms of the sum in the form A * (10**6)**B
    Qsum = 0
    
    jList = np.array(range(abs(L-l2),l3-abs(L-l1)+1,2))


    a_sum1, b_sum1 =  Reduced_E(l1,L,l3-jList) # Reduced, because E contains an if statement that is usually True anyway.
    a_sum2, b_sum2 =  Reduced_E(l2,L,jList) #

        
    Q_part = ((-k2/k1).reshape(len(k1),1))**jList /(a_sum1*a_sum2)

    
    Qsum = Q_part.dot((mp.mpf((10**6)))**(b_pre-b_sum2-b_sum1)) #THESE TWO LINES TAKE MOST TIME. For l = 100 it takes 153 of 190 seconds.

    

    
    
    return prefactor*Qsum


In [9]:
def J_Modified(l1,l2,l3,k1,k2,k3):
    
#Teta is equal to 1 anyway, if we choose alpha and beta accordingly.

    
    mu = ((k1**2+k2**2-k3**2)/(2*k1*k2))
    
    


    prefactor = np.pi/((2*k1*k2))*((k1/k3))**l3 *1/k3 #*Teta(k1,k2,k3)
    

    Lsum = mp.mpf(1.)*np.array([0.]*len(prefactor))

    Lower = (l1+l2-l3)//2
    
    for L in range(Lower,(l1+l2+l3)//2+1):
        
        Mult = Q_Modified(l3,L,k1,l1,k2,l2)
        
        #unfortunately, mp.legendre doesnt accept arrays as arguments :-(
        
        Legendrearr = np.array([mp.legendre(L,mu_entry) for mu_entry in mu])

        Lsum += Mult*Legendrearr

    return prefactor * Lsum  

In [10]:
def ReduceArrays(Alphas,Betas):
    # This function validates the tetrahedron condition for the alpha and beta meshgrid.
    
    
    Valid_Indices = []
    
    for i in range(len(Alphas)):
        if(Alphas[i]<=1-Betas[i]):
            if(Alphas[i]>=-(1-Betas[i])):
                Valid_Indices.append(i)
    
    
    return Valid_Indices

In [11]:
def Int_G_Meshgrid(alpha_array,beta_array,l1,l2 = 0,l3 = 0):
    #Currently only accepts numpy arrays as alpha/beta array.
    #returns the integral over the 3 bessel functions. Analytically.
    
    #What I am currently doing: Check that The Alphas and Betas fulfill the "triangle condition", i.e. -(1-b)<=alph<=1-b
    #then calculate the integral with the remaining ones, then put every remaining pair of alpha/beta to zero.
    
    if(l2 == 0 or l3 == 0):
        l2 = l1
        l3 = l1
    
    shape = alpha_array.shape
    
    alpha_array = alpha_array.flatten()
    beta_array = beta_array.flatten()

    Indices  = ReduceArrays(alpha_array,beta_array)

    
    
    red_alpha_array = alpha_array[Indices]
    red_beta_array = beta_array[Indices]
    
    #converts the arrays to mpfloats
    a_arr = mp.mpf(1.)*(1-red_beta_array)
    b_arr = mp.mpf(1.)*1./2*(1+red_alpha_array+red_beta_array)
    c_arr = mp.mpf(1.)*1./2*(1-red_alpha_array+red_beta_array)
    
    
    Integral_Array = J_Modified(l1,l2,l3,a_arr,b_arr,c_arr)
    
    Values = np.zeros(len(alpha_array))
    
    Values[Indices] = Integral_Array #this one overwrites the zeros with the actual values at the right position.
    
    Values = Values.reshape(shape)
    
    return Values


In [12]:
beta_initial = np.linspace(0.0001,0.999,100) # Here, one can define the density of sampling in the alpha/beta plane.
alpha_initial = np.linspace(-1,1,100) 

AlphaArray, BetaArray = np.meshgrid(alpha_initial,beta_initial)



In [13]:
XMax = 80000.    #XMax = 1000. for standard accuracy
XSteps = 8000000  #XSteps = 10000 for standard accuracy
Stepwidth = XMax/XSteps #This is the numerical integral of the Bessel functions for comparison purposes.
x = np.arange(0,XMax,Stepwidth)



def Int_G_Num(alpha,beta,l,x_sample = np.arange(0,1000,10000),Valueonly = True):
    
    a = np.array(1-beta)
    b = np.array((1+alpha+beta)/2.)
    c = np.array((1-alpha + beta)/2.)
    
    BesA = Bessel(np.tensordot(a,x_sample,0),l)
    BesB = Bessel(np.tensordot(b,x_sample,0),l)
    BesC = Bessel(np.tensordot(c,x_sample,0),l)
    
    Stepwidth = x_sample[3]-x_sample[2] #assuming linearly spaced x


    if (Valueonly == True):
        return Stepwidth*((x**2).dot((BesA*BesB*BesC).transpose()))
    
    return Stepwidth*((x**2).dot((BesA*BesB*BesC).transpose()))#,x_sample,x_sample**2*BesA*BesB*BesC




In [21]:
#This one will calculate values for individual coices of l1,l2,l3; k1,k2,k3
l1 = 6
l2 = 8
l3 = 8
k1 = 0.4 #exemplary values.
k2 = 0.7
k3 = 0.9
Single_Point_Data = J_Modified(l1,l2,l3,mp.mpf(1.)*np.array([k1]),mp.mpf(1.)*np.array([k2]),mp.mpf(1.)*np.array([k3]))
#Note that points can only be handed over in arrays AND must be mpfloats.

    
#This one will calculate whole Tetrahedron-cut-throughs
Data = Int_G_Meshgrid(AlphaArray,BetaArray,l1,l2,l3) #Calculates the integral for a tetrahedron cut-through at l1,l2,l3

[mpf('-1.05449306979951174521663266542052827810757805534479906778155467803379148611654118424968091393425587862318078147335752404471057312904217739795434616258035368329664131214980575951049690986768095935883409')]


In [22]:
np.save("DataDreieckMashgridMyData.npy",Data) #Save the Data with the prefix "DataDreieckMashgrid", so it will be plotted.